## 1. Install SpamAssassin

In [5]:
!docker pull clarity99/spamassasin

Using default tag: latest
latest: Pulling from clarity99/spamassasin

b959ec00: Pulling fs layer 
c583622e: Pulling fs layer 
6d330a19: Pulling fs layer 
1e81812b: Pulling fs layer 
54a58020: Pulling fs layer 
66651efd: Pulling fs layer 
Digest: sha256:811533d8aa6ba5bb866ceb78f816f5cb1d3af22c3360f70bef93f4f1f1ffdacc[6ADownloading  8.747MB/45.34MBExtracting     283B/283B
Status: Downloaded newer image for clarity99/spamassasin:latest
docker.io/clarity99/spamassasin:latest


In [6]:
import pandas as pd
import os

df = pd.read_csv("emails.csv") 
print(df.head())

                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1


In [7]:
from email.utils import formatdate
import uuid

os.makedirs("emls", exist_ok=True)
eml_paths = []
for idx, row in df.iterrows():
    eml_path = f"emls/email_{idx}.eml"
    with open(eml_path, "w") as f:
        f.write(
            f"Date: {formatdate()}\n"
            f"Message-ID: <{uuid.uuid4()}@example.com>\n"
            f"Subject: Test Email\n"
            f"From: sender@example.com\n"
            f"To: recipient@example.com\n"
            f"Content-Type: text/plain\n\n"
            f"{row['text']}"
        )
    eml_paths.append(eml_path)

df["eml_path"] = eml_paths

In [ ]:
import subprocess

def spamassassin_predict(eml_path):
    """Run SpamAssassin in Docker and return 1 (spam) or 0 (ham)."""
    try:
        result = subprocess.run(
            [
                "docker", "run", "--rm",
                "-v", f"{os.getcwd()}:/data",
                "clarity99/spamassasin",
                "spamassassin", "-t", f"/data/{eml_path}"
            ],
            capture_output=True, text=True
        )
        return 1 if "X-Spam-Status: Yes" in result.stdout else 0
    except:
        return -1  

df["spamassassin_pred"] = df["eml_path"].apply(spamassassin_predict)